# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

import os
import joblib
import pandas as pd
import sys

from sklearn.model_selection import train_test_split




## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl'

experiment=Experiment(ws, experiment_name)



Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R2ADRGRDY to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

datastore AzureBlob
workspaceblobstore AzureBlob
workspacefilestore AzureFile


In [4]:
# df = Dataset.get_by_name(ws, name='corona-nlp-train').to_pandas_dataframe()
df = pd.read_csv("Corona_NLP_train.csv")

In [5]:
# for col in df.columns:
#      if "Col" in col:
#          df = df.drop(col)

In [6]:
df.head()

,Unnamed: 0,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,0,3799,48751.0,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,3800,48752.0,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,2,3801,48753.0,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,3802,48754.0,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,4,3803,48755.0,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [7]:
df.Sentiment.unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive', nan], dtype=object)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41159 entries, 0 to 41158
Data columns (total 7 columns):
Unnamed: 0       41158 non-null object
UserName         41158 non-null object
ScreenName       41157 non-null float64
Location         32567 non-null object
TweetAt          41157 non-null object
OriginalTweet    41157 non-null object
Sentiment        41155 non-null object
dtypes: float64(1), object(6)
memory usage: 2.2+ MB


In [42]:
df_modified = df[["OriginalTweet", "Sentiment"]].copy()
df_modified["OriginalTweet"] = df_modified["OriginalTweet"].astype("str")
df_modified["Sentiment"] = df_modified["Sentiment"].astype("str")
df_modified = df_modified[df_modified["Sentiment"] != "nan"]

       
positive = df_modified["Sentiment"].isin(["Neutral", "Positive", 'Extremely Positive'])
negative = df_modified["Sentiment"].isin(['Extremely Negative', 'Negative'])

df_modified.loc[positive, "Sentiment"] = 1
df_modified.loc[negative, "Sentiment"] = 0

df_modified["Sentiment"] = df_modified["Sentiment"].astype("int")

In [43]:
df_modified["Sentiment"].unique()

array([1, 0])

In [41]:
df_modified.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,advice Talk to your neighbours family to excha...,1
2,Coronavirus Australia: Woolworths to give elde...,1
3,My food stock is not the only one which is emp...,1
4,"Me, ready to go at supermarket during the #COV...",0


In [44]:
df_modified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41155 entries, 0 to 41158
Data columns (total 2 columns):
OriginalTweet    41155 non-null object
Sentiment        41155 non-null int64
dtypes: int64(1), object(1)
memory usage: 964.6+ KB


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [45]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes":30,
    "task":"classification",
    "primary_metric":"accuracy",
    "training_data":df_modified,
    "label_column_name":"Sentiment",
    "n_cross_validations":3}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [46]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

Running on local machine
Parent Run ID: AutoML_8f672ae7-d852-40a4-8672-d266a69580be

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************************************************************************

WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Connection pool is full, discarding connection: mlstrg122217.blob.core.windows.net
WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9bf8a73f28>: Failed to establish a new connection: [Errno 111] Connection refused',)': /azureml/ExperimentRun/dcid.AutoML_8f672ae7-d852-40a4-8672-d266a69580be_0/outputs/internal_cross_validated_models.pkl?sv=2019-02-02&sr=b&sig=w93CIyoiy3UmV8sFOUHAabgng%2

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [47]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [48]:
best_run_automl, best_model_automl = automl_run.get_output()
best_run_metrics = best_run_automl.get_metrics()
parameter_values = best_run_automl.get_details()

print("Best Run Id: ", best_run_automl.id)
print("\n")
print("\n")
print("Accuracy: ", best_run_metrics["accuracy"])
print("\n")
print("\n")
print("Parameters: ", parameter_values)
print("\n")
print("\n")
print("Metrics for best run: ", best_run_automl.get_metrics())

Best Run Id:  AutoML_8f672ae7-d852-40a4-8672-d266a69580be_7




Accuracy:  0.7250155415844688




Parameters:  {'runId': 'AutoML_8f672ae7-d852-40a4-8672-d266a69580be_7', 'status': 'Completed', 'startTimeUtc': '2020-10-22T16:27:53.063113Z', 'endTimeUtc': '2020-10-22T16:29:41.892126Z', 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-automl\',\'compute_target\':\'local\',\'subscription_id\':\'13fe1ad9-91fc-4300-9ca0-27ebe69e0476\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_8f672ae7-d852-40a4-8672-d266a69580be_7","experiment_name":"capstone-automl","workspace_name

In [49]:
# referencce - https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.run.automlrun?view=azure-ml-py
print(best_model_automl.steps[-1])

('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                 

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [50]:
# Personal ToDo - Check why can't register model from get_output() directly
best_model_automl = best_run_automl.register_model(model_name="capstone-automl.pkl", model_path="./")

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service